In [1]:
import pandas as pd
import numpy as np
import re as re

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

In [2]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#df reading

df_ventas = pd.read_csv("D:/Desktop/items_ordered_2years.csv", sep=",")

En este dataframe los duplicados son valores que no tienen valor, puesto que no puede haber dos productos con el mismo item_id. Por tanto, son valores que hay que eliminar

In [4]:
#Drop duplicates
df_ventas = df_ventas.drop_duplicates()

df_ventas

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
0,ce30c2f02458457e3c7b563a636ae2a1,0916c05c5c3f65f59d813a78ac35c8d2,2018-11-06 16:52:13,86434,1.0,9.3750,15.38,8.0,eccbc87e4b5ce2fe28308fd9f2a7baf3,TARAZONA DE LA MANCHA,02100
1,ce30c2f02458457e3c7b563a636ae2a1,ff323b39ae36843396d2e53ce549fb10,2018-11-06 16:52:13,87652,1.0,7.0000,15.41,8.0,eccbc87e4b5ce2fe28308fd9f2a7baf3,TARAZONA DE LA MANCHA,02100
2,ce30c2f02458457e3c7b563a636ae2a1,199916dffc95259f4d2daab6664ca9c0,2018-11-06 16:52:13,2785,1.0,9.1587,12.23,8.0,eccbc87e4b5ce2fe28308fd9f2a7baf3,TARAZONA DE LA MANCHA,02100
3,83e75d608f11c8163599806420903ab9,8ca334ec2493501139327ce0165a1a84,2018-12-17 12:26:54,13000,5.0,13.7936,15.03,50.0,eccbc87e4b5ce2fe28308fd9f2a7baf3,TARAZONA DE LA MANCHA,02100
4,9b687270d8e7eed9717022af5961a190,ce0a1683c6b1a0248b330344ec592ddf,2017-01-12 14:19:03,41947,1.0,5.5924,7.00,5.0,1679091c5a880faf6fb5e6087eb1b2dc,Alboraya,46120
...,...,...,...,...,...,...,...,...,...,...,...
930955,479ed436c26a511c4c3356e679bccbc9,37d700d54a6acc0ff49392cce34b6427,2018-12-30 23:51:57,1868,1.0,4.0700,4.79,10.0,d9cd4886b615a36059f72a74d7019113,Ubeda,23400
930956,479ed436c26a511c4c3356e679bccbc9,05fd04261f1c951b07f2ac3a67254d81,2018-12-30 23:51:57,23539,1.0,4.0700,5.35,10.0,d9cd4886b615a36059f72a74d7019113,Ubeda,23400
930957,bf923aa16e0608a5ed95347a9f411dce,8d8600ea4e451330f8614ba08ad4f047,2018-12-30 23:57:37,92793,1.0,33.7360,32.93,10.0,d73f27d3ab96df71f0fdc47511f326bc,Vic,08500
930958,bf923aa16e0608a5ed95347a9f411dce,b74b2e1a096c656a247142e24eefb738,2018-12-30 23:57:37,93036,1.0,0.0000,16.32,10.0,d73f27d3ab96df71f0fdc47511f326bc,Vic,08500


In [5]:
df_ventas.nunique()

num_order           278045
item_id             906317
created_at          277896
product_id           26396
qty_ordered             57
base_cost            52202
price                 5028
discount_percent        45
customer_id         118799
city                 21012
zipcode              11132
dtype: int64

Sigue habiendo mas registros que valores únicos en item_id, lo cual indica que hay que revisar qué puede estar generando esa duplicidad de id.

### ITEM_ID

In [6]:
df_ventas["item_id"].nunique()

906317

In [7]:
item_nu = df_ventas.groupby(
    "item_id", as_index=False
).size()

mascara = item_nu.iloc[:,1] > 1

item_repetidos = item_nu[mascara]

item_repetidos

,item_id,size
111,000802dde903f8282df756d2aab8d024,2
135,000a8c9981c4dc672abd207bdc1b85f7,2
236,001183d1253fa3d736eda6c92c826360,2
375,001b62e0e6c6fcf7fd771422bc91cb12,2
385,001bd90aa12b86fc11c4a416b3495cd5,2
...,...,...
906117,fff249858b2d160946da7b8613aa15da,2
906126,fff28e48c4498626a0add97bb3c9cb55,2
906144,fff3cf8f9ffdc3188bfb0b0546b37245,2
906292,fffe4633ea7afca58a2a1e68f2963f6b,2


In [8]:
mascara2 = df_ventas["item_id"].isin(
    item_repetidos["item_id"].tolist()
)

df_ventas_repetido = df_ventas[mascara2].sort_values("item_id")

df_ventas_repetido

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
649241,8051708bf244ba8518b5e5b61a74486b,000802dde903f8282df756d2aab8d024,2018-12-17 23:32:08,7647,1.0,2.5438,3.00,5.0,c2d3e675fedd5555ee14b6441f0a8568,CORRALES DE BUELNA,39400
649259,8051708bf244ba8518b5e5b61a74486b,000802dde903f8282df756d2aab8d024,2018-12-17 23:32:08,7647,1.0,2.5438,3.00,5.0,c2d3e675fedd5555ee14b6441f0a8568,Corrales de buelna,39400
447200,6d19dedf8710788ea7d774c3fc977a9b,000a8c9981c4dc672abd207bdc1b85f7,2018-09-02 16:35:36,3076,2.0,6.3986,12.71,15.0,9bd04a4cc151456b49c699624bb83084,Granada,18009
447182,6d19dedf8710788ea7d774c3fc977a9b,000a8c9981c4dc672abd207bdc1b85f7,2018-09-02 16:35:36,3076,2.0,6.3986,12.71,15.0,9bd04a4cc151456b49c699624bb83084,Granada,18009
766220,27df73640a5bbe494d1db3e3bf16b906,001183d1253fa3d736eda6c92c826360,2018-08-07 09:29:02,2021,1.0,10.2800,15.87,5.0,0a6f053fb763ab1d4b6bef36ce683e53,barcelona,08025
...,...,...,...,...,...,...,...,...,...,...,...
370897,bfffc8aa56b415ede4538a4a022c771a,fff3cf8f9ffdc3188bfb0b0546b37245,2017-09-18 18:50:26,2503,2.0,5.0720,6.57,7.0,b6b909c1193aba68c413cc1442743ac7,MADRID,28002
91194,d69c15626b6406bbcc1384407e75a075,fffe4633ea7afca58a2a1e68f2963f6b,2018-10-31 09:23:06,3290,1.0,4.1466,5.59,8.0,91ae48805e8e96055d454f043054059b,Alameda de la Sagra,45240
91255,d69c15626b6406bbcc1384407e75a075,fffe4633ea7afca58a2a1e68f2963f6b,2018-10-31 09:23:06,3290,1.0,4.1466,5.59,8.0,91ae48805e8e96055d454f043054059b,PANTOJA,45290
803228,4f8b96d2253a9d4a6ad4672b4e228c48,ffff1e771c21fcbfed80ea59dcb7bf5d,2018-04-23 20:52:23,68375,1.0,5.2100,6.95,5.0,a99855ff1ad46db840df3a48bf075d17,CANGAS,36940


Las duplicidades del item_id aparecen debido a presumiblemente errores en el zipcode/city. Se tratará despues de corregir las columnas de city y zipcode

### NULOS

In [9]:
df_ventas.isna().sum()

num_order              0
item_id                0
created_at             0
product_id             0
qty_ordered            0
base_cost           2358
price                  0
discount_percent       0
customer_id            0
city                2910
zipcode             2910
dtype: int64

Valores nulos en zipcode y city similares. Pueden ser coincidentes.

In [10]:
mascara_na = df_ventas["zipcode"].notna()
df_ventas[mascara_na ].isna().sum()


num_order              0
item_id                0
created_at             0
product_id             0
qty_ordered            0
base_cost           2355
price                  0
discount_percent       0
customer_id            0
city                   0
zipcode                0
dtype: int64

Efectivamente, al filtrar por zipcodes que no tuvieran na en la columa zipcode desaparecían los valores perdidos en la de city.
Esos registros dificilmente pueden ser imputados por algoritmos como randomforest o knn, sin arriesgarse a estar añadiendo información de forma casi aleatoria.
Por ello, se prescindirá de esos registros.

In [11]:
df_ventas = df_ventas[mascara_na]

### ZIPCODE

In [12]:
df_ventas["longitud_zip"] = df_ventas["zipcode"].dropna().apply(len)
df_ventas["longitud_zip"].unique().tolist()

C:\Users\Daniel\AppData\Local\Temp\ipykernel_2572\704870013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas["longitud_zip"] = df_ventas["zipcode"].dropna().apply(len)


[5,
 2,
 8,
 4,
 6,
 7,
 3,
 15,
 10,
 9,
 1,
 14,
 17,
 11,
 12,
 16,
 19,
 21,
 24,
 18,
 22,
 13,
 20,
 25,
 29,
 26,
 27,
 28,
 23,
 31]

Funciones para limpieza de city y zipcode

In [13]:
#Limpieza de zipcodes con RegEx
def num_guion(string):
    """ Get a string with the numbers and hyphens of another string
    
    Args:
        df: string used to extract the string with numbers abd hyphens

    Returns:
        df: the string with numbers and hyphens
    """
    aux = re.match("([\d-]+)", str(string))
    try:
        return str(aux.group())
    except:
        return string


# Limpieza de nombres de ciudad

# def AcentosLimpiador(text):
# 	acentos = {'ñ':'n','á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'à':'a'}
# 	for ele in acentos:
# 		if ele in text:
# 			text = text.replace(ele, acentos[ele])
# 	return text

# def CityCleaner(text):
#     stopWordSpanish = set(stopwords.words('spanish'))
#     wordTokens = word_tokenize(AcentosLimpiador(text.lower()).rstrip()) 
#     filteredSentence = [element for element in wordTokens if not element in stopWordSpanish] 
#     return filteredSentence

    


Limpieza de ambas columnas para facilitar el merge con la tabla de zipcodes

In [14]:
# Limpieza zipcode
df_ventas["zipcode"] = df_ventas["zipcode"].apply(lambda x: num_guion(x))
# df_ventas["city"] = df_ventas["city"].apply(lambda x: CityCleaner(x))
# df_ventas["city"] = df_ventas["city"].apply(lambda x: "".join(x))

C:\Users\Daniel\AppData\Local\Temp\ipykernel_2572\2691356738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas["zipcode"] = df_ventas["zipcode"].apply(lambda x: num_guion(x))
C:\Users\Daniel\AppData\Local\Temp\ipykernel_2572\2691356738.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas["city"] = df_ventas["city"].apply(lambda x: CityCleaner(x))
C:\Users\Daniel\AppData\Local\Temp\ipykernel_2572\2691356738.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

Se carga el dataframe con la información de privincias, países, zipcodes y ciudades.

In [24]:
zipcodes_csv = pd.read_csv("DataScrapped.csv")
zipcodes_csv.drop("Unnamed: 0", axis=1, inplace=True)
zipcodes_csv.drop_duplicates(inplace=True)


In [25]:
zipcodes_csv

,Country,Region,City,Zipcode
0,Spain,Castilla - La Mancha,Tarazona De La Mancha,02100
4,Spain,Comunidad Valenciana,Alboraya,46120
74,Spain,Comunidad Valenciana,Benidorm,03503
76,Spain,Castilla - La Mancha,Yunquera De Henares,19210
88,Spain,Cataluna,Cornella De Llobregat,08940
...,...,...,...,...
929904,Portugal,Porto,Grijo,4415-430
929965,Portugal,Lisboa,Mem Martins,2725-590
930087,Portugal,Lisboa,Lisboa,2620-459
930101,Portugal,Porto,Rio Tinto,4435-644


Se genera una copia del dataframe con las columnas city y zipcode para un primer join/merge

In [26]:
# Nombres de las columnas a minuscula

column_list = zipcodes_csv.columns.tolist()
for i in range(len(column_list)):
    column_list[i] = column_list[i].lower()
zipcodes_csv.columns = column_list

# Obtener dataframe con solo city y zipcode, y pasar city a minusculas para facilitar el match

# zipcodes_to_join = zipcodes_csv.loc[:, ["zipcode", "city"]]
# zipcodes_to_join["city"] = zipcodes_to_join["city"].apply(lambda x: CityCleaner(x))
# zipcodes_to_join["city"] = zipcodes_to_join["city"].apply(lambda x: "".join(x))

Para obtener aquellos zipcodes que estén mal escritos pero que estén bien la ciudad, se realiza un primer merge por columna city

In [36]:

df_ventas.drop("city", inplace=True, axis=1)

KeyError: 'zipcode_x'

In [49]:
df_ventas_2 = df_ventas.merge(zipcodes_csv, how="left", on="zipcode")

In [50]:
df_ventas_2

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city_x,zipcode,longitud_zip,country,region,city_y
0,ce30c2f02458457e3c7b563a636ae2a1,0916c05c5c3f65f59d813a78ac35c8d2,2018-11-06 16:52:13,86434,1.0,9.3750,15.38,8.0,eccbc87e4b5ce2fe28308fd9f2a7baf3,tarazonamancha,02100,5,Spain,Castilla - La Mancha,Tarazona De La Mancha
1,ce30c2f02458457e3c7b563a636ae2a1,ff323b39ae36843396d2e53ce549fb10,2018-11-06 16:52:13,87652,1.0,7.0000,15.41,8.0,eccbc87e4b5ce2fe28308fd9f2a7baf3,tarazonamancha,02100,5,Spain,Castilla - La Mancha,Tarazona De La Mancha
2,ce30c2f02458457e3c7b563a636ae2a1,199916dffc95259f4d2daab6664ca9c0,2018-11-06 16:52:13,2785,1.0,9.1587,12.23,8.0,eccbc87e4b5ce2fe28308fd9f2a7baf3,tarazonamancha,02100,5,Spain,Castilla - La Mancha,Tarazona De La Mancha
3,83e75d608f11c8163599806420903ab9,8ca334ec2493501139327ce0165a1a84,2018-12-17 12:26:54,13000,5.0,13.7936,15.03,50.0,eccbc87e4b5ce2fe28308fd9f2a7baf3,tarazonamancha,02100,5,Spain,Castilla - La Mancha,Tarazona De La Mancha
4,9b687270d8e7eed9717022af5961a190,ce0a1683c6b1a0248b330344ec592ddf,2017-01-12 14:19:03,41947,1.0,5.5924,7.00,5.0,1679091c5a880faf6fb5e6087eb1b2dc,alboraya,46120,5,Spain,Comunidad Valenciana,Alboraya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912250,479ed436c26a511c4c3356e679bccbc9,37d700d54a6acc0ff49392cce34b6427,2018-12-30 23:51:57,1868,1.0,4.0700,4.79,10.0,d9cd4886b615a36059f72a74d7019113,ubeda,23400,5,Spain,Andalucia,Ubeda
912251,479ed436c26a511c4c3356e679bccbc9,05fd04261f1c951b07f2ac3a67254d81,2018-12-30 23:51:57,23539,1.0,4.0700,5.35,10.0,d9cd4886b615a36059f72a74d7019113,ubeda,23400,5,Spain,Andalucia,Ubeda
912252,bf923aa16e0608a5ed95347a9f411dce,8d8600ea4e451330f8614ba08ad4f047,2018-12-30 23:57:37,92793,1.0,33.7360,32.93,10.0,d73f27d3ab96df71f0fdc47511f326bc,vic,08500,5,Spain,Cataluna,Vic
912253,bf923aa16e0608a5ed95347a9f411dce,b74b2e1a096c656a247142e24eefb738,2018-12-30 23:57:37,93036,1.0,0.0000,16.32,10.0,d73f27d3ab96df71f0fdc47511f326bc,vic,08500,5,Spain,Cataluna,Vic


In [52]:
df_ventas_2.isna().sum()


num_order              0
item_id                0
created_at             0
product_id             0
qty_ordered            0
base_cost           2355
price                  0
discount_percent       0
customer_id            0
city_x                 0
zipcode                0
longitud_zip           0
country             1616
region              1616
city_y              1616
dtype: int64

In [46]:
df_ventas_3.isna().sum()

num_order               0
item_id                 0
created_at              0
product_id              0
qty_ordered             0
base_cost           30103
price                   0
discount_percent        0
customer_id             0
longitud_zip            0
zipcode                 0
country               580
region                580
city                  580
dtype: int64

In [47]:
df_ventas_2 = df_ventas_3[mascara2].sort_values("item_id")

df_ventas_2

C:\Users\Daniel\AppData\Local\Temp\ipykernel_2572\739567747.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_ventas_3 = df_ventas_3[mascara2].sort_values("item_id")


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).